# Importing Libraries

In [1]:
import torch
#from torch import nn
#import torch.nn.functional as F
from torchvision import datasets, transforms

#import numpy as np

#import requests

import matplotlib as mpl
#import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
#from mpl_toolkits.axes_grid1.inset_locator import inset_axes

#from PIL import Image, ImageOps

#import reprlib
import sys

In [2]:
x = torch.tensor([[[1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]],
                  [[1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]]])
x.shape

torch.Size([2, 4, 4])

In [3]:
x.unsqueeze(0).unsqueeze(1).shape

torch.Size([1, 1, 2, 4, 4])

# Configuring Visualization Parameters

In [4]:
%matplotlib inline

In [5]:
XINHUI = "#7a7374"
XUEBAI = "#fffef9"
YINBAI = "#f1f0ed"
YINHUI = "#918072"

figure_size = (16, 9)

In [6]:
custom_params = {
    "axes.axisbelow": True,
    "axes.edgecolor": YINBAI,
    "axes.facecolor": XUEBAI,
    "axes.grid": True,
    "axes.labelcolor": XINHUI,
    "axes.spines.right": False,
    "axes.spines.top": False,
    "axes.titlecolor": XINHUI,
    "figure.edgecolor": YINBAI,
    "figure.facecolor": XUEBAI,
    "grid.alpha": .8,
    "grid.color": YINBAI,
    "grid.linestyle": "--",
    "grid.linewidth": 1.2,
    "legend.edgecolor": YINHUI,
    "patch.edgecolor": XUEBAI,
    "patch.force_edgecolor": True,
    "text.color": XINHUI,
    "xtick.color": YINHUI,
    "ytick.color": YINHUI,
}

mpl.rcParams.update(custom_params)

# Configuring Other Notebook Parameters

In [7]:
#reprlib_rules = reprlib.Repr()
#reprlib_rules.maxother = 250

# Pre-installing Custom Functions

In [8]:
sys.path.append("../")

In [9]:
from Modules import *

In [10]:
def im_convert(tensor):
    image = tensor.cpu().clone().detach().numpy()
    image = image.transpose(1, 2, 0)
    image = image * np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5))
    image = image.clip(0, 1)
    return image


def image_to_scalar_data(tensor):
    image = tensor.cpu().clone().detach().numpy()
    image = image.transpose(1, 2, 0)
    image = image.reshape(image.shape[0], -1)
    image = image * 0.5 + 0.5
    image = image.clip(0, 1)
    return image

# Practicing in Stages

## Testing LeNet

In [11]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

training_dataset = datasets.CIFAR10(root="../Datasets",
                                    train=True,
                                    download=True,
                                    transform=transform)

validation_dataset = datasets.CIFAR10(root="../Datasets",
                                      train=False,
                                      download=True,
                                      transform=transform)

training_loader = torch.utils.data.DataLoader(training_dataset,
                                              batch_size=100,
                                              shuffle=True)

validation_loader = torch.utils.data.DataLoader(validation_dataset,
                                                batch_size=100,
                                                shuffle=False)

tabulation = Form_Generator()
tabulation.heading_printer(
    "Loading and transformation of the CIFAR10 training and validation datasets"
)

statements = [
    """
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

training_dataset = datasets.CIFAR10(root="../Datasets",
                                    train=True,
                                    download=True,
                                    transform=transform)

validation_dataset = datasets.CIFAR10(root="../Datasets",
                                      train=False,
                                      download=True,
                                      transform=transform)

training_loader = torch.utils.data.DataLoader(training_dataset,
                                              batch_size=100,
                                              shuffle=True)

validation_loader = torch.utils.data.DataLoader(validation_dataset,
                                                batch_size=100,
                                                shuffle=False)
"""
]
tabulation.statement_generator(statements)

variables = [
    "transform",
    "training_dataset",
    "validation_dataset",
]
values = [
    str(transform),
    str(training_dataset),
    str(validation_dataset),
]
tabulation.variable_generator(variables, values)

expressions = [
    "len(training_loader)", "len(training_loader.dataset)",
    "list(training_loader)[0][0].shape", "list(training_loader)[0][1].shape",
    "next(iter(training_loader))[0].shape",
    "next(iter(training_loader))[1].shape", "len(validation_loader)",
    "len(validation_loader.dataset)", "list(validation_loader)[0][0].shape",
    "list(validation_loader)[0][1].shape",
    "next(iter(validation_loader))[0].shape",
    "next(iter(validation_loader))[1].shape"
]
results = [
    str(len(training_loader)),
    str(len(training_loader.dataset)),
    str(list(training_loader)[0][0].shape),
    str(list(training_loader)[0][1].shape),
    str(next(iter(training_loader))[0].shape),
    str(next(iter(training_loader))[1].shape),
    str(len(validation_loader)),
    str(len(validation_loader.dataset)),
    str(list(validation_loader)[0][0].shape),
    str(list(validation_loader)[0][1].shape),
    str(next(iter(validation_loader))[0].shape),
    str(next(iter(validation_loader))[1].shape),
]
tabulation.expression_generator(expressions, results, 12)

Files already downloaded and verified
Files already downloaded and verified
Loading and transformation of the CIFAR10 training and validation datasets

    +-------------------------------------------------------------+
    | Statement                                                   |
    +-------------------------------------------------------------+
    | transform = transforms.Compose([                            |
    |     transforms.Resize((32, 32)),                            |
    |     transforms.ToTensor(),                                  |
    |     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  |
    | ])                                                          |
    |                                                             |
    | training_dataset = datasets.CIFAR10(root="../Datasets",     |
    |                                     train=True,             |
    |                                     download=True,          |
    |                           

In [12]:
len(training_loader)

500

In [13]:
training_dataset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ../Datasets
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )